## 1. 什么是XGBoost

XGBoost 所应用的算法就是 gradient boosting decision tree，既可以用于分类也可以用于回归问题中。

**那什么是 Gradient Boosting？**

Gradient boosting 是 boosting 的其中一种方法

所谓 Boosting ，就是将弱分离器 $f_i(x)$ 组合起来形成强分类器 $F(x)$ 的一种方法。

所以 Boosting 有三个要素：

* A loss function to be optimized：例如分类问题中用 cross entropy，回归问题用 mean squared error。

* A weak learner to make predictions：例如决策树。

* An additive model：将多个弱学习器累加起来组成强学习器，进而使目标损失函数达到极小。

Gradient boosting 就是通过加入新的弱学习器，来努力纠正前面所有弱学习器的残差，最终这样多个学习器相加在一起用来进行最终预测，准确率就会比单独的一个要高。之所以称为 Gradient，是因为在添加新模型时使用了梯度下降算法来最小化的损失。

XGBoost 高效地实现了 GBDT 算法并进行了算法和工程上的许多改进。
XGBoost本质上还是一个GBDT，但是力争把速度和效率发挥到极致，所以叫X (Extreme) GBoosted。包括前面说过，两者都是boosting方法。

关于GBDT，可以查看前一篇的介绍，[点此跳转](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/Gradient%20Boosting%20Decision%20Tree.ipynb)。

一般来说，gradient boosting 的实现是比较慢的，因为每次都要先构造出一个树并添加到整个模型序列中。

而 XGBoost 的特点就是计算速度快，模型表现好，这两点也正是这个项目的目标。

表现快是因为它具有这样的设计：

* Parallelization：训练时可以用所有的 CPU 内核来并行化建树。
* Distributed Computing ：用分布式计算来训练非常大的模型。
* Out-of-Core Computing：对于非常大的数据集还可以进行 Out-of-Core Computing。
* Cache Optimization of data structures and algorithms：更好地利用硬件。


### 1.1 XGBoost树的定义

先来举个**例子**，我们要预测一家人对电子游戏的喜好程度，考虑到年轻和年老相比，年轻更可能喜欢电子游戏，以及男性和女性相比，男性更喜欢电子游戏，故先根据年龄大小区分小孩和大人，然后再通过性别区分开是男是女，逐一给各人在电子游戏喜好程度上打分，如下图所示。
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/20.png?raw=true" width="500"/>

就这样，训练出了 2 棵树 tree1 和 tree2，类似之前 gbdt 的原理，两棵树的结论累加起来便是最终的结论，所以小孩的预测分数就是两棵树中小孩所落到的结点的分数相加：2 + 0.9 = 2.9。爷爷的预测分数同理：-1 + （-0.9）= -1.9。具体如下图所示：
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/21.png?raw=true" width="520"/>

恩？这不是跟上文介绍的 GBDT 异曲同工么？

事实上，如果不考虑工程实现、解决问题上的一些差异，XGBoost 与 GBDT **比较大的不同就是目标函数的定义**。XGBoost 的目标函数如下图所示：
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/22.png?raw=true" width="500"/>

其中：

- 红色箭头所指向的 $l$ 即为损失函数（比如平方损失函数：$l(y_i,y^i)=(y_i-y^i)^2$
- 红色方框所框起来的是正则项（包括 L1 正则、L2 正则）
- 红色圆圈所圈起来的为常数项
- 对于 f(x)，XGBoost 利用泰勒展开三项，做一个近似。**f(x)表示的是其中一颗回归树。**
- 最终的目标函数只依赖于每个数据点的在误差函数上的一阶导数和二阶导数。

**在这里只做一个简要式的讲解，具体的算法细节和公式求解请查看这篇博文，讲得很仔细**：
* [通俗理解kaggle比赛大杀器xgboost](https://blog.csdn.net/v_JULY_v/article/details/81410574)
* [XGBoost原理](https://blog.csdn.net/ye1215172385/article/details/79590698)

XGBoost 的**核心算法思想**不难，基本就是：

1. 不断地添加树，不断地进行特征分裂来生长一棵树，每次添加一个树，其实是学习一个新函数 **f(x)**，去拟合上次预测的残差。
2. 当我们训练完成得到 k 棵树，我们要预测一个样本的分数，其实就是根据这个样本的特征，在每棵树中会落到对应的一个叶子节点，每个叶子节点就对应一个分数
3. 最后只需要将每棵树对应的分数加起来就是该样本的预测值。

显然，我们的目标是要使得树群的预测值 $y_i^{'}$ 尽量接近真实值 $y_i$，而且有尽量大的泛化能力。类似之前 GBDT 的套路，XGBoost 也是需要将多棵树的得分累加得到最终的预测得分（每一次迭代，都在现有树的基础上，增加一棵树去拟合前面树的预测结果与真实值之间的残差）。
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/23.png?raw=true" width="500"/>

那接下来，我们如何选择每一轮加入什么 f 呢？答案是非常直接的，选取一个 f 来使得我们的目标函数尽量最大地降低。这里 f 可以使用泰勒展开公式近似。
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/24.png?raw=true" width="500"/>

实质是把样本分配到叶子结点会对应一个 obj，优化过程就是 obj 优化。也就是分裂节点到叶子不同的组合，不同的组合对应不同 obj，所有的优化围绕这个思想展开。到目前为止我们讨论了目标函数中的第一个部分：训练误差。接下来我们讨论目标函数的第二个部分：正则项，即如何定义树的复杂度。

### 1.2 正则项：树的复杂度

XGBoost对树的复杂度包含了两个部分：

- 一个是树里面叶子节点的个数 T
- 一个是树上叶子节点的得分 w 的 L2 模平方（对 w 进行 L2 正则化，相当于针对每个叶结点的得分增加 L2 平滑，目的是为了避免过拟合）
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/25.png?raw=true" width="500"/>

我们再来看一下 XGBoost 的目标函数（损失函数揭示训练误差 + 正则化定义复杂度）：

$L(\phi)=\sum_{i}l(y_i^{'}-y_i)+\sum_k\Omega(f_t)$

正则化公式也就是目标函数的后半部分，对于上式而言，$y_i^{'}$ 是整个累加模型的输出，正则化项 $∑kΩ(ft)$ 是则表示树的复杂度的函数，值越小复杂度越低，泛化能力越强。

### 1.3 树该怎么长

很有意思的一个事是，我们从头到尾了解了 xgboost 如何优化、如何计算，但树到底长啥样，我们却一直没看到。很显然，一棵树的生成是由一个节点一分为二，然后不断分裂最终形成为整棵树。那么树怎么分裂的就成为了接下来我们要探讨的关键。对于一个叶子节点如何进行分裂，XGBoost 作者在其原始论文中给出了一种分裂节点的方法：**枚举所有不同树结构的贪心法**

不断地枚举不同树的结构，然后利用打分函数来寻找出一个最优结构的树，接着加入到模型中，不断重复这样的操作。这个寻找的过程使用的就是**贪心算法**。选择一个 feature 分裂，计算 loss function 最小值，然后再选一个 feature 分裂，又得到一个 loss function 最小值，你枚举完，找一个效果最好的，把树给分裂，就得到了小树苗。

总而言之，XGBoost 使用了和CART回归树一样的想法，利用贪婪算法，遍历所有特征的所有特征划分点，不同的是使用的目标函数不一样。具体做法就是分裂后的目标函数值比单子叶子节点的目标函数的增益，同时为了限制树生长过深，还加了个阈值，只有当增益大于该阈值才进行分裂。从而继续分裂，形成一棵树，再形成一棵树，**每次在上一次的预测基础上取最优进一步分裂/建树。**

### 1.4 如何停止树的循环生成

凡是这种循环迭代的方式必定有停止条件，什么时候停止呢？简言之，设置树的最大深度、当样本权重和小于设定阈值时停止生长以防止过拟合。具体而言，则

1. 当引入的分裂带来的增益小于设定阀值的时候，我们可以忽略掉这个分裂，所以并不是每一次分裂 loss function 整体都会增加的，有点预剪枝的意思，阈值参数为（即正则项里叶子节点数 T 的系数）；
2. 当树达到最大深度时则停止建立决策树，设置一个超参数 max_depth，避免树太深导致学习局部样本，从而过拟合；
3. 样本权重和小于设定阈值时则停止建树。什么意思呢，即涉及到一个超参数-最小的样本权重和 min_child_weight，和 GBM 的 min_child_leaf 参数类似，但不完全一样。大意就是一个叶子节点样本太少了，也终止同样是防止过拟合；

## 2. XGBoost 与GBDT 有什么不同

除了算法上与传统的 GBDT 有一些不同外，XGBoost 还在工程实现上做了大量的优化。总的来说，两者之间的区别和联系可以总结成以下几个方面。 

1. GBDT 是机器学习算法，XGBoost 是该算法的工程实现。
2. 在使用 CART 作为基分类器时，XGBoost 显式地加入了正则项来控制模 型的复杂度，有利于防止过拟合，从而提高模型的泛化能力。
3. GBDT 在模型训练时只使用了代价函数的一阶导数信息，XGBoost 对代 价函数进行二阶泰勒展开，可以同时使用一阶和二阶导数。
4. 传统的 GBDT 采用 CART 作为基分类器，XGBoost 支持多种类型的基分类器，比如线性分类器。
5. 传统的 GBDT 在每轮迭代时使用全部的数据，XGBoost 则采用了与随机森林相似的策略，支持对数据进行采样。
6. 传统的 GBDT 没有设计对缺失值进行处理，XGBoost 能够自动学习出缺失值的处理策略。

## 3. 为什么 XGBoost 要用泰勒展开，优势在哪里？

XGBoost 使用了一阶和二阶偏导, 二阶导数有利于梯度下降的更快更准. 使用泰勒展开取得函数做自变量的二阶导数形式, 可以在不选定损失函数具体形式的情况下, 仅仅依靠输入数据的值就可以进行叶子分裂优化计算, 本质上也就把损失函数的选取和模型算法优化/参数选择分开了. 这种去耦合增加了 XGBoost 的适用性, 使得它按需选取损失函数, 可以用于分类, 也可以用于回归。

## 4. XGBoost 参数调节 (sklearn风格接口)

[XGBoost类库使用小结](https://www.cnblogs.com/pinard/p/11114748.html)

#### XGBoost 框架参数:
对于XGBoost的框架参数，最重要的是3个参数: 

* **booster [default = gbtree]** 
 * 每次迭代用的模型，有两种，但基本上都是使用树模型。:
  * gbtree: 基于树的模型 (CART决策树)
  * gblinear: 线性模型

* **n_estimators [default = 100]**
 * 非常重要的要调的参数，它关系到 XGBoost 的复杂度，因为它代表了决策树弱学习器的个数。这个参数对应 sklearn GBDT 的 n_estimators。n_estimators 太小，容易欠拟合，n_estimators 太大，模型会过于复杂，一般需要调参选择一个适中的数值。(一般 n_estimators 和 learning_rate 两个参数一起调节才有效果)

* **objective [default = reg:squarederror]**
 * 在回归问题 objective 一般使用 reg:squarederror ，即MSE均方误差。二分类问题一般使用 binary:logistic, 多分类问题一般使用 multi:softmax。
 * 试图使 loss function 最小化:
  * binary:logistic –logistic regression for binary classification, returns predicted probability (not class) 返回预测的概率。
  * multi:softmax –multiclass classification using the softmax objective, returns predicted class (not probabilities) 返回预测的类别。
   * 在这种情况下，你还需要多设一个参数：num_class(类别总数目)。
  * multi:softprob –same as softmax, but returns predicted probability of each data point belonging to each class. 返回的是每个数据记录属于各个类别的概率。

#### XGBoost 每个弱学习器参数: 
一般先调 max_depth，min_child_weight 和 gamma。如果发现有过拟合的情况下，再尝试调后面几个参数。

* **max_depth [default = 6]**
 * 树的最大深度，数据少或者特征少的时候可以不管这个值。如果模型样本量多，特征也多的情况下，需要限制这个最大深度，具体的取值一般要网格搜索 (GridSeachCV) 调参。
 * 也是用于控制过拟合 (over-fitting)，树越深模型会学到更具体的样本。如果树的深度太大会导致过拟合。
 * Typical values: 3-10

* **min_child_weight [default = 1]**
 * 最小的子节点权重阈值，如果某个树节点的权重小于这个阈值，则不会再分裂子树，即这个树节点就是叶子节点。这里树节点的权重使用的是该节点所有样本的二阶导数的和：
   * $H_{tj} =  \sum\limits_{x_i \in R_{tj}}h_{ti}$
 * 这个参数用于避免过拟合 (over-fitting)。当它的值较大时，可以避免模型学习到局部的特殊样本。
 * 但是如果这个值过高，会导致欠拟合。这个值需要网格搜索 (GridSeachCV) 寻找最优值。

* **gamma [default = 0]**
 * 在节点分裂时，只有分裂后损失函数 (loss function) 的下降值超过 gamma，才会分裂这个节点。gamma指定了节点分裂，所需的损失函数下降的最低阈值。如果 gamma 设置为0，表示只要使得 loss 函数减少，就分裂。
 * 这个参数的值越大，算法越保守。**属于需要调整的重点参数**。
 * 这个值需要网格搜索 (GridSeachCV) 寻找最优值。
 * 范围[0,∞]


 
* **subsample [default = 1]**
 * 这个参数在构建树时候，每棵树对数据集采样的百分比（不放回采样）。eg：如果设为 0.8，表示随机抽取样本中80%的个体来构建树。
 * 减小这个参数的值，算法会更加保守，避免过拟合。这个值设置得过小，它可能会导致欠拟合。（因为采样过小）。因此取值不能太低。
 * 初期可以取值 1，如果发现过拟合后可以网格搜索 (GridSeachCV) 调参找一个相对小一些的值。
 * 一般取值 0.5 到 1。

* **colsample_bytree / colsample_bylevel / colsample_bynode**
 * 这三个参数都是用于特征采样的，默认都是不做采样，即使用所有的特征建立决策树。
  * colsample_bytree 控制整棵树的特征采样比例
  * colsample_bylevel 控制某一层的特征采样比例，
  * colsample_bynode 控制某一个树节点的特征采样比例
 * 比如我们一共 64 个特征，则假设 colsample_bytree，colsample_bylevel 和 colsample_bynode 都是 0.5，则某一个树节点分裂时会随机采样 8 个特征来尝试分裂子树。
 
* **reg_lambda [default = 1]**
 * L2 正则项系数 (L2 regularization term on weight)
 * 增大 L2 正则项的系数，使得模型保守。避免过拟合 (over-fitting)
 
* **reg_alpha [default = 0]**
 * L1 正则项系数 (L1 regularization term on weight)
 * 应用在很高维度的情况下，可以使得算法的速度更快。
 
* **scale_pos_weight [default = 1]**
 * 在各类别样本十分不平衡时，把这个参数设定为一个正值，可以使算法更快收敛。

#### 其他：
主要是 learning_rate。

* **learning_rate [default = 0.1]**
 * 控制每个弱学习器的权重缩减系数。较小的 learning_rate 需要更多的弱学习器的迭代次数。通常用步长和迭代最大次数一起来决定算法的拟合效果。所以这两个参数 n_estimators 和learning_rate 要一起调节才有效果。
 * 当然也可以先固定一个 learning_rate ，然后调完 n_estimators，再调完其他所有参数后，最后再来调 learning_rate 和 n_estimators。
 * 一般常用的数值: 0.01-0.2
 
* **silent [default = 1]**
 * 设为 1 则不打印执行信息。设为 0 打印信息
 
* **n_jobs [default to maximum number of threads available if not set]**
 * 控制算法的并发线程数
 * 如果你希望使用 CPU 全部的核，那就不要输入这个参数 (None)，算法会自动检测它。
 
* **max_delta_step [default = 0]**
 * 这参数限制每棵树权重改变的最大步长。如果这个参数的值为0，那就意味着没有约束。如果它被赋予了某个正值，那么它会让这个算法更加保守。
 * 但是，这个参数一般不需要设置。但是当各类别的样本十分不平衡的时候，需要设置。

* **eval_metric [ default according to objective ]**
 * The metric to be used for validation data.
 * 对于回归问题默认采用rmse，对于分类问题一般采用error
 * Typical values are:
  * rmse – root mean square error
  * mae – mean absolute error
  * logloss – negative log-likelihood
  * error – Binary classification error rate (0.5 threshold)
  * merror – Multiclass classification error rate
  * mlogloss – Multiclass logloss
  * auc: Area under the curve

## 6. [代码实现]()